In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import psycopg2
import uuid

In [3]:
driver = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver2 = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')

In [4]:
driver.get("https://esports-betonline.ultraplay.net/esports/today")
driver2.get("https://esports-betonline.ultraplay.net/esports/program/0")

sleep(5)
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
html2 = driver2.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")
soup2 = BeautifulSoup(html2, "html.parser")


In [5]:
# get soup
#print(soup)

In [6]:
# get leagues

sportList = soup.find_all('div',{'class': 'panel panel-default panel-sport hidden-lg ng-scope'})

#print(sportList[0])

for sport in sportList:
    #print(sportList)
    sportName = sport.find('div',{'class': 'row panel-heading border-none'}).text.strip()
    
    print(sportName)
    
    leagueList = sport.find_all('div', {'class': 'panel panel-default margin-bottom-none panel-league ng-scope'})
    
    for league in leagueList:
        leagueName = league.find('div', {'class': 'col-xs-10 col-sm-11 text-left panel-heading match-detail-league-name'}) 
        if leagueName is not None:
            print(leagueName.text.strip())

Counter-Strike
ECS Season 4
MakeMy.Bet Cup
LOOT.BET Cup 2
CSGO.NET Cup
LOOT.BET Nations Elite Esports Cup
Dota 2
MID.TV Dota2
Overwatch
Nexus Cup
World of Tanks
The Final Battle
Gwent
CHALLENGER 2


In [19]:
# connect to postgresql
conn = psycopg2.connect(dbname="BetOnline", user="Admin1", password="1407athNBC!", host = 'betonlinedb.cfitahvq3xgc.us-west-1.rds.amazonaws.com')
cur = conn.cursor()

In [29]:
todayOdds = pd.DataFrame(columns=['OddsID','GameID','TeamID','Moneyline','DatePulled','LookupID_O'])
todatMatches = pd.DataFrame(columns=['MatchID','Sport','League','GameID','TeamA','TeamB','Date','LookupID_G'])
todayTeams = pd.DataFrame(columns=['TeamID','TeamName','Sport','League','LookupID_T'])
todayLeagues = pd.DataFrame(columns=['LeagueID','TeamName','Sport','League','LookupID_L'])
todaySports = pd.DataFrame(columns=['SportID','SportName'])

i = 0

sportList = soup.find_all('div',{'class': 'panel panel-default panel-sport visible-lg ng-scope'})
for sport in sportList:
    sportName = sport.find('div',{'class': 'col-xs-6 ng-binding'}).text.strip()

    # look up sport ID, otherwise add it
    print('inputting'+sportName)
    SportID = sportDBCheck(sportName, conn)
    
    leagueList = sport.find_all('div', {'class': 'panel panel-default margin-bottom-none panel-league ng-scope'})
    for league in leagueList:
        leagueName = league.find('div', {'class': 'col-xs-10 col-sm-11 text-left panel-heading match-detail-league-name'}).text.strip() 
             
        gameList = league.find_all('div', {'class': 'panel-match-detail ng-scope'})
        
        for game in gameList:
            teamA = game.find('div', {'class':'match-detail-teamA-row'})
            teamB = game.find('div', {'class':'match-detail-teamB-row'})
            
            if teamA is not None:
                teamName1 = teamA.find('div', {'class': 'match-detail-team-name'}).text.strip()
                teamName2 = teamB.find('div', {'class': 'match-detail-team-name'}).text.strip()
                Odds1 = teamA.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})
                Odds2 = teamB.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})

                if 'suspended' not in (game.get('class')):
                    ML1 = Odds1[0].text.strip()
                    ML2 = Odds2[0].text.strip()

                # create dataframe to hold entry
                pdOdds = pd.DataFrame([[0, 0, 0, ML1, 0, 0],[0, 0, 0, ML2, 0, 0]],
                                      columns=['UUID_O','GameID','TeamID','Moneyline','DatePulled','LookupID_O'])
                
                pdGames = pd.DataFrame([[0, sportName, leagueName, 0, 0, 0, 0, 0]],
                                      columns=['UUID_G','Sport','League','GameID','TeamA','TeamB','Date','LookupID_G'])
                    
                pdTeams = pd.DataFrame([[0, teamName1, sportName, leagueName, 0], [0, teamName2, sportName, leagueName, 0]],
                                      columns=['UUID_T','TeamName','Sport','League','LookupID_T'])             

                todayOdds = todayOdds.append(pdOdds)                
                todayGames = todatMatches.append(pdGames)
                todayTeams = todayTeams.append(pdTeams)
                i = i+1
#print(todayTeams)

inputtingCounter-Strike
query = INSERT INTO "BetSport" ("SportID","SportName") VALUES (UUID('08a082a5-1a8e-4a1e-a656-eb85814766da'), 'Counter-Strike')
inputtingDota 2
query = INSERT INTO "BetSport" ("SportID","SportName") VALUES (UUID('cf065a32-09c4-4aa7-b251-981a5b7e6175'), 'Dota 2')
inputtingOverwatch
query = INSERT INTO "BetSport" ("SportID","SportName") VALUES (UUID('18dd9d3e-e0dc-4a10-9544-cb903c51ac1c'), 'Overwatch')
inputtingWorld of Tanks
query = INSERT INTO "BetSport" ("SportID","SportName") VALUES (UUID('219b50df-1984-4dab-be78-8041825f6fae'), 'World of Tanks')
inputtingGwent
query = INSERT INTO "BetSport" ("SportID","SportName") VALUES (UUID('450abd12-185a-4cff-8547-8717d30247a7'), 'Gwent')


In [28]:
# define helper functions

# return sport ID if it exists, otherwise create a new entry and return sportID
def sportDBCheck (sportName, connection):
    sportDF = pd.read_sql('SELECT * FROM "BetSport"', con=connection)
    try: 
        SportID = sportDF[sportDF['SportName'] == sportName].iloc[0]['SportID']
        print('found sport name')
    except:
        SportID =  uuid.uuid4() #str(uuid.uuid4()) 
        queryStart = 'INSERT INTO "BetSport" ("SportID","SportName") VALUES '
        queryEnd = [SportID,sportName]
        query = queryStart + str(tuple(queryEnd))
        
        print('query = ' + query)
        try:
            cur.execute(query)
        except Exception as e: 
            print(e)
            #print("error committing sport sql code")
            pass
        conn.commit()

    return SportID
        




In [18]:
cur.close()
conn.close()